In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import string
import re

base_dir = Path("C:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis")
data_dir = base_dir / "raw_data" # raw_data contains kaggle data with data quality defects from messy package
output_dir = base_dir / "Dataset" # Dataset contains cleaned data

In [2]:
def clean_str_cols(df, exclude=[]):
    cut = df[exclude].copy()
    df = df.drop(columns=exclude)

    for column in df.select_dtypes(include=["object"]).columns:

        # Remove punctuation
        df[f"{column}"] = df[f"{column}"].str.replace(
            f"[{re.escape(string.punctuation)}]", 
            "",
            regex=True)
        
        df[f"{column}"] = df[f"{column}"].str.title() # Standardize Capitalization
        df[f"{column}"] = df[f"{column}"].str.strip() # Remove leading and trailing spaces 

    
    df = df.join(cut)

    return df

def clean_date_cols(df, date_cols):
    for col in date_cols:

        df[f"{col}"] = df[f"{col}"].str.replace(
            f"[{re.escape(string.punctuation.replace("/", ""))}]",
            "",
            regex=True
        )

        df[f"{col}"] = df[f"{col}"].str.strip()

    return df

In [3]:
pd.set_option("display.max_rows", None)

### Clean Accounts.csv

In [4]:
# os.chdir(data_dir) # Ensure we are in the raw_data dir

# df = pd.read_csv("accounts.csv") 

# df = clean_str_cols(df, exclude=["created_at"]) # Clean str columns

# # Change floating cols to
# df['account_id'] = df['account_id'].astype('Int64')
# df['mrr'] = df['mrr'].astype('Int64')

# # Save the cleaned accounts to Dataset dir
# os.chdir(output_dir)
# df.to_parquet(output_dir / "accounts.parquet", index=False)

# # Switch back to raw_data for the next table cleaning
# os.chdir(data_dir)

### Clean customers.csv

In [5]:
# df = pd.read_csv("customers.csv")

# df['customer_id'] = df['customer_id'].astype('Int64')

# df['signup_date'] = df['signup_date'].str.strip()

# df['monthly_fee'] = df['monthly_fee'].astype('Int64')

# df['acquisition_cost'] = df['acquisition_cost'].astype('Int64')

# df['churn_date'] = df['churn_date'].str.strip()
# df['churn_date'] = df['churn_date'].str.lower()

# df.to_parquet(output_dir / "customers.parquet", index=False)

### Clean events.csv

In [6]:
# os.chdir(data_dir)
# df = pd.read_csv("events.csv")

# df['event_id'] = df['event_id'].astype('Int64')
# df['user_id'] = df['user_id'].astype('Int64')


# df = df.drop(columns='...6')

# df = clean_str_cols(df, exclude=['occurred_at'])

# df['occurred_at'] = df['occurred_at'].str.strip()

# df.to_parquet(output_dir / "events.parquet", index=False)

In [7]:
for csv in data_dir.glob("*.csv"):
    print(csv.name)

accounts.csv
customers.csv
events.csv
experiments.csv
googleplaystore_user_reviews.csv
revenue.csv
subscriptions.csv
users.csv
variants.csv
variant_exposures.csv


### Clean experiments.csv

In [8]:
# os.chdir(data_dir)

# df = pd.read_csv("experiments.csv")

# # df.head(5)

# # df.dtypes

# df['experiment_id'] = df['experiment_id'].astype("Int64")

# df = clean_str_cols(df, exclude=['start_date', 'end_date'])

# df = clean_date_cols(df, date_cols=['start_date', 'end_date'])

# df.to_parquet(output_dir / "experiments.parquet")


### Clean googleplaystore_user_reviews.csv


In [ ]:
# os.chdir(data_dir)

# df = pd.read_csv("googleplaystore_user_reviews.csv")

In [10]:
for i in data_dir.glob("*.csv"):
    print(i.name)

accounts.csv
customers.csv
events.csv
experiments.csv
googleplaystore_user_reviews.csv
revenue.csv
subscriptions.csv
users.csv
variants.csv
variant_exposures.csv


In [ ]:
# # pd.options.display.float_format = "{:.2f}".format

# # df.head()

# # df.dtypes

# df = clean_str_cols(df)

# df['Sentiment'] = (df['Sentiment'].str.lower()).astype('str')

# df['Sentiment_Polarity'] = df['Sentiment_Polarity'] \
# .astype("str") \
# .where(df['Sentiment_Polarity'].notna()) \
# .str.replace(r"^0", "0.", regex=True) \
# .astype("float")

In [ ]:
# df['Sentiment_Subjectivity'] = df['Sentiment_Subjectivity'] \
# .astype("str") \
# .where(df['Sentiment_Polarity'].notna()) \
# .str.replace(r"^0", "0.", regex=True) \
# .astype("float")

In [ ]:
# df = df.drop(columns=['App', 'Translated_Review'])

# df.to_parquet(output_dir / "googleplaystore_user_reviews.parquet")